In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

In [6]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1,28,28,1).astype('float32')/255.0
x_test = x_test.reshape(-1,28,28,1).astype('float32')/255.0

In [7]:
x_train.shape, x_test.shape

((60000, 28, 28, 1), (10000, 28, 28, 1))

In [8]:
y_train.shape, y_test.shape

((60000,), (10000,))

In [11]:
class CNNBlock(layers.Layer):
    def __init__(self, out_channels, kernel_size=3):
        super(CNNBlock, self).__init__()
        self.conv = layers.Conv2D(out_channels, kernel_size, padding='same')
        self.bn = layers.BatchNormalization()
        
    
    def call(self, input_tensor, training=False):
        x = self.conv(input_tensor)
        x = self.bn(x, training = training)
        x = tf.nn.relu(x)
        return x
    
model = keras.Sequential(
    [
        CNNBlock(32), 
        CNNBlock(64),
        CNNBlock(128),
        layers.Flatten(),
        layers.Dense(10),
    ]
)

model.compile(
    optimizer = keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
)

model.fit(x_train,  y_train, batch_size=64, epochs=3)
model.evaluate(x_test, y_test, batch_size=64)

Epoch 1/3
938/938 [==============================] - 407s 433ms/step - loss: 0.6186 - accuracy: 0.9449
Epoch 2/3
938/938 [==============================] - 428s 456ms/step - loss: 0.1023 - accuracy: 0.9806
Epoch 3/3
157/157 [==============================] - 15s 90ms/step - loss: 0.0630 - accuracy: 0.9824


[0.06300148367881775, 0.9824000000953674]

In [15]:
class ResBlock(layers.Layer):
    def __init__(self,channels):
        super(ResBlock, self).__init__()
        self.cnn1 = CNNBlock(channels[0])
        self.cnn2 = CNNBlock(channels[1])
        self.cnn3 = CNNBlock(channels[2])
        self.pooling = layers.MaxPooling2D()
        self.identity_mapping = layers.Conv2D(channels[1], 3, padding='same')
        
    
    def call(self, input_tensor, training=False):
        x = self.cnn1(input_tensor, training=training)
        x = self.cnn2(x, training=training)
        x = self.cnn3(x + self.identity_mapping(input_tensor), training=training)
        return self.pooling(x)
    
    
class ResNet_Like(keras.Model):
    def __init__(self, num_classes = 10):
        super(ResNet_Like, self).__init__()
        self.block1 = ResBlock([32, 32, 64])
        self.block2 = ResBlock([128, 128, 256])
        self.block3 = ResBlock([256, 256, 512])
        self.pool = layers.GlobalAveragePooling2D()
        self.classifier = layers.Dense(num_classes)
        
    
    def call(self, input_tensor, training=False):
        x = self.block1(input_tensor, training=training)
        x = self.block2(x, training=training)
        x = self.block3(x, training=training)
        x = self.pool(x)
        return self.classifier(x)
    
    
model = ResNet_Like(num_classes=10)
model.compile(
    optimizer = keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
)

model.fit(x_train,  y_train, batch_size=64, epochs=1)
model.evaluate(x_test, y_test, batch_size=64)

157/157 [==============================] - 41s 259ms/step - loss: 0.1706 - accuracy: 0.9504


[0.17064660787582397, 0.9503999948501587]

In [16]:
model.summary()

Model: "res_net__like_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 res_block_6 (ResBlock)      multiple                  28896     
                                                                 
 res_block_7 (ResBlock)      multiple                  592512    
                                                                 
 res_block_8 (ResBlock)      multiple                  2954496   
                                                                 
 global_average_pooling2d_2   multiple                 0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_4 (Dense)             multiple                  5130      
                                                                 
Total params: 3,581,034
Trainable params: 3,577,706
Non-trainable params: 3,328
_____________________________________